# Obtaining the best redshift from 3 Catalogs: Griffiths, 3dHST, UltraVISTA

## What we have:
## Griffiths has spec-z, photo-z. Their best Z is 'high-quality spec z else photoz.' It is not known how high-quality is defined. The errors listed on their photoz are 3sig. 
## 3dHST has spec-zs, photo-zs, and grism-zs. Their 'z_best' can be any of these.
## UltraVISTA just has photo-zs. 

## Workflow for choosing best redshift:
## 1) If Griffiths spec-z = Griffiths best-z, then use that. 
## else if 3dHST spec-z = 3dHST best_z, use that. 
## else (no good spec-z's anywhere):
## compare 1-sig errors on Griffiths photoz, 3dHST grism z, 3dHST photoz, UltraVISTA photoz. choose that with lowest err. Note: all except Griffiths report 1 sig errors; Griffiths reports 3sig, so that error is divided by 3 in the comparison.

 



In [4]:
%matplotlib inline

from astropy.io import fits as pyfits
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from astropy.table import Table,Column
from operator import itemgetter
from datetime import datetime
import requests
import os

In [6]:
def download_from_dropbox(url):
    
    local_filename = "{:}".format(url.split("/")[-1].split("?")[0])
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                f.flush()
            
    return local_filename

In [7]:
old_catalog_filename = download_from_dropbox("https://www.dropbox.com/s/ge7sgtf0crxmwyi/all_redshift_info.fits?dl=1")
old_catalog = Table.read(old_catalog_filename) 

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:120: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:120: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


In [8]:
subjects=set(old_catalog['OBJNO'])


In [42]:
updated_table=Table()
for col_name in old_catalog.columns.names:
    c0 = Column([x[col_name] for x in old_catalog], name=col_name) 
    updated_table.add_columns([c0])


In [43]:
def get_best_redshift(galaxy):
    
    #define Griffiths variables
    griffiths_specz = galaxy['SPECZ']
    griffiths_bestz = galaxy['Z']
    griffiths_photoz = galaxy['PHOTOZ']
    griffiths_photoz_err_3sig = galaxy['PHOTOZ_ERR']/galaxy['PHOTOZ']
    griffiths_photoz_err_1sig = galaxy['PHOTOZ_ERR']/galaxy['PHOTOZ']/3.

    
    #define 3dhst variables
    dhst_specz = galaxy['z_spec']
    dhst_bestz = galaxy['z_best']
    dhst_photoz = galaxy['z_peak_phot']
    dhst_photoz_err = (galaxy['z_phot_u68']-galaxy['z_phot_l68'])/dhst_photoz

    dhst_grismz = galaxy['z_max_grism']
    dhst_grismz_err = (galaxy['z_grism_u68']-galaxy['z_grism_l68'])/dhst_grismz

    
    if str(dhst_photoz_err)=='nan':
        dhst_photoz_err = 99
    if dhst_photoz < 0:
        dhst_photoz_err = 99
    if str(dhst_grismz_err)=='nan':
        dhst_grismz_err = 99
    if dhst_grismz < 0:
        dhst_grismz_err = 99
    
    #define ultravista variables
    uv_photoz = galaxy['zPDF']
    uv_photoz_err = (galaxy['zPDF_u68']-galaxy['zPDF_l68'])/uv_photoz
    if str(uv_photoz_err)=='nan':
        uv_photoz_err = 99
    if uv_photoz < 0:
        uv_photoz_err=99
    
    #Order of operations to determine which redshift to use:
    #1) If specz exists in Griffiths catalog, use that first. 
    if griffiths_specz == griffiths_bestz and griffiths_specz > 0 and griffiths_specz < 9:
        return griffiths_specz,'Griffiths'
        
    #if no specz in Griffiths, check 3Dhst
    else:
        if dhst_specz == dhst_bestz and dhst_specz >0 and dhst_specz <9: 
            return dhst_bestz, '3DHST'
        
        else: #no speczs, so check which photoz has smallest error. 
            photoz_list = [[griffiths_photoz,griffiths_photoz_err_1sig,'Griffiths'],[dhst_photoz,dhst_photoz_err,'3DHST'],[uv_photoz,uv_photoz_err,['UltraVISTA']]]
            photoz_list = sorted(photoz_list, key=itemgetter(1)) #sort by which has smallest error            
            
            return photoz_list[0][0],photoz_list[0][2]

In [44]:
for galaxy in updated_table:
    galaxy['Z_BEST'], galaxy['Z_BEST_SOURCE'] = get_best_redshift(galaxy)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: RuntimeWarning: divide by zero encountered in float_scalars
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:8: RuntimeWarning: divide by zero encountered in float_scalars
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: RuntimeWarning: invalid value encountered in float_scalars
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:8: RuntimeWarning: invalid value encountered in float_scalars


In [45]:
fname = '/home/user/mel/Dropbox/gzhubble/hubble_files/gzh_redshift_data/best_redshift_table_%i_%i_%i.fits'%(datetime.now().month,datetime.now().day,datetime.now().year)

if os.path.exists(fname):
    os.remove(fname)
updated_table.write(fname,format='fits')


In [57]:
galaxy = old_catalog[5]

In [22]:
    dhst_specz = galaxy['z_spec']
    dhst_bestz = galaxy['z_best']
    dhst_photoz = galaxy['z_peak_phot']


In [58]:
(galaxy['zPDF_u68']-galaxy['zPDF_l68'])/galaxy['zPDF']

0.68181813

In [59]:
(galaxy['z_grism_u68']-galaxy['z_grism_l68'])/galaxy['z_max_grism']

nan

In [60]:
(galaxy['z_phot_u68']-galaxy['z_phot_l68'])/galaxy['z_peak_phot']

nan

In [61]:
galaxy['PHOTOZ_ERR']/galaxy['PHOTOZ']/3.

0.1111111044883728

In [62]:
 galaxy['PHOTOZ_ERR']/galaxy['PHOTOZ']

0.33333331

In [63]:
galaxy['PHOTOZ']

0.0099999998